# 패키지 설치
pip 명령어로 의존성 있는 패키지를 설치합니다.



In [10]:
import os, glob

In [1]:
!pip install -e git+https://github.com/judy3378/ratsnlp.git@get_trainer#egg=ratsnlp

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Obtaining ratsnlp from git+https://github.com/judy3378/ratsnlp.git@get_trainer#egg=ratsnlp
  Updating ./src/ratsnlp clone (to revision get_trainer)
  Running command git fetch -q --tags
  Running command git reset --hard -q 77f257e7a223817964e063f1497ad749800bb726
  Preparing metadata (setup.py) ... done
DEPRECATION: pytorch-lightning 1.6.1 has a non-standard dependency specifier torch>=1.8.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Attempting uninstall: ratsnlp
    Found existing installation: ratsnlp 1.0.53
    Uninstalling ratsnlp-1.0.53:
      Successfully uninstalled ratsnlp-1.0.53
  Running setup.py develop for ratsnlp
Note: you may need to restart the kernel to use upda

# 각종 설정
모델 하이퍼파라메터(hyperparameter)와 저장 위치 등 설정 정보를 선언합니다.

In [1]:
from ratsnlp.nlpbook.classification.arguments import ClassificationDeployArguments
args = ClassificationDeployArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_model_dir="test07",
    max_seq_length=256,
)

/root/miniconda3/envs/intern12/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


downstream_model_checkpoint_fpath: test07/epoch=1-val_loss=0.14.ckpt


# 모델 로딩
파인튜닝을 마친 모델과 토크나이저를 읽어 들입니다.

In [2]:
import torch
from transformers import BertConfig, BertForSequenceClassification
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device("cpu")
)
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel(),
)
model = BertForSequenceClassification(pretrained_model_config)
model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [3]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

# 인퍼런스 함수 선언
인퍼런스 함수를 선언합니다.

In [7]:
def read_text_from_file(file_path):
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                sentence = file.read()
                return sentence
        except FileNotFoundError:
            print("File not found. Please check the file path.")
            return None
        except Exception as e:
            print(f"An error occurred: {e}")
            return None

In [8]:
def inference_fn(sentence):

        inputs = tokenizer( 
            [sentence], 
            max_length=args.max_seq_length,
            padding="max_length",
            truncation=True,
        )

        class_names = ["뉴스", "역사기록물", "보고서", "사설"]

        with torch.no_grad():
            outputs = model(**{k: torch.tensor(v) for k, v in inputs.items()})
            prob = outputs.logits.softmax(dim=1)
            probs = [round(p.item(), 4) for p in prob[0]]
            pred_class_index = torch.argmax(prob)
            pred = class_names[pred_class_index]

        return {
            #'sentence': sentence,
            'prediction': pred,
            'probs': {class_names[i]: probs[i] for i in range(len(class_names))},
        }

In [13]:
txt_files_folder = 'documents'  # Folder containing the text files
txt_files_pattern = os.path.join(txt_files_folder, '*.txt')
txt_file_paths = glob.glob(txt_files_pattern)

In [14]:
for file_number, txt_file_path in enumerate(txt_file_paths, start=1):
    sentence = read_text_from_file(txt_file_path)
    
    if sentence is not None:
        file_name = os.path.basename(txt_file_path)  # Extract the file name from the full path
        result = inference_fn(sentence)
        print(f"File {file_number}: {file_name}")  # Print the file number and name
        print(result)
        print("=" * 30)

IndexError: list index out of range

In [5]:
def read_text_from_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            sentence = file.read()
            return sentence
    except FileNotFoundError:
        print("File not found. Please check the file path.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

txt_file_path = 'Documents/briefing.txt'  # 파일 경로를 적절하게 수정하세요.
sentence = read_text_from_file(txt_file_path)

if sentence is not None:
    print("Content of the file:")
    print(sentence)


Content of the file:
행안부 장관, 연일 세계잼버리

안전 및 폭염 대응 위한 현장점검

 - 행안부·문체부 장관, 전북도지사 및 경찰·소방·군 등 참여

 - 폭염대비 물품 지급, 식사의 질·양 및 의료서비스 제공 등 논의

 
□ 이상민 행정안전부 장관은 오늘도 잼버리 현장을 찾아 긴급점검회의를 열고 잼버리 지원대책을 논의하였다.

 ○ 이번 회의에서는 현장 상황을 정확하고 즉각적으로 파악할 수 있도록 공동 조직위원장인 행안부·문체부 장관과 전라북도지사뿐만 아니라 현장을 지원 중인 경찰·소방 인력 및 지원을 제공 중인 군 인력 등이 모두 참석하였다.

 ○ 오늘 회의에서 이상민 장관은 어제 논의된 지원방안 추진상황을 점검하는 한편, 추가 지원 필요사항을 논의하였다. 이어서, 잼버리 조직위원회, 전북소방본부와 전북경찰청이 8월 6일(일) K-POP 콘서트 준비상황을 차례로 보고하였다.

□ 이 자리에서 이상민 장관은 오늘부로 참가자 전원에게 충분한 생수와 얼음을 지급하고, 개인용 폭염대비 물품도 차질없이 지급되도록 챙겨달라고 주문했다.

 ○ 또한, 참가자들이 제때, 충분히, 그리고 단 한명도 누락되지 않고 이를 전달받을 수 있도록 현장에서 꼼꼼히 확인하라고 지시하였다.

 ○ 더불어 참가자들에게 충분하고 영양가 높은 식사가 제공될 수 있도록 오늘 즉시 조치해달라고 말했다.

 
□ 이 장관은 생수의 효과적 비치 및 보급을 위한 냉동탑차와 아이스박스의 충원 등 필수물자를 신속하게 보급할 것과,

 ○ 추가 운영·설치되는 냉방(쿨링)버스 230대와 그늘막 이용에 어려움이 없도록 하고, 덩굴터널 조명 등을 설치하여 야간에도 더욱 안전하게 생활할 수 있도록 즉시 조치할 것을 주문하였다.

 ○ 또한, 온열질환자 등 치료가 필요한 부상자들에게 의료 서비스가 적시에 이루어질 수 있도록 의료인력 증원과 의료품 추가 보급에도 각별히 유의해달라고 강조했다.

□ 더불어, 참가자들이 안내받지 못해 필요한 지원을 받지 못하는 일이 없도록 적극적으로 안내할 것을

In [6]:
def inference_fn(sentence):

    inputs = tokenizer( 
        [sentence], 
        max_length=args.max_seq_length,
        padding="max_length",
        truncation=True,
    )

#    print(inputs.items())
#    print(inputs.keys())
#   print(inputs.values())

#    inputs = tokenizer(
#            [sentence],
#        max_length=args.max_seq_length,
#        padding="max_length",
#        truncation=True,
#    )

    class_names = ["뉴스", "보도자료", "역사기록물", "보고서", "회의록", "사설", "간행물", "연설문", "문학", "나래이션"]

    with torch.no_grad():
        outputs = model(**{k: torch.tensor(v) for k, v in inputs.items()})
        prob = outputs.logits.softmax(dim=1)
        probs = [round(p.item(), 4) for p in prob[0]]
        pred_class_index = torch.argmax(prob)
        pred = class_names[pred_class_index]

    return {
        #'sentence': sentence,
        'prediction': pred,
        'probs': {class_names[i]: probs[i] for i in range(len(class_names))},
    }

result = inference_fn(sentence)
print(result)

{'prediction': '연설문', 'probs': {'뉴스': 0.0205, '보도자료': 0.3573, '역사기록물': 0.0079, '보고서': 0.0063, '회의록': 0.0016, '사설': 0.0019, '간행물': 0.0048, '연설문': 0.5981, '문학': 0.001, '나래이션': 0.0006}}
